In [1]:
import cv2
import numpy as np
import imutils
import os
import shutil
from matplotlib import pyplot as plt
from IPython.display import Image 
from PIL import Image        
from types import SimpleNamespace
import random as rng
from skimage import io, morphology, img_as_bool, segmentation
from scipy import ndimage as ndi
import pprint
import itertools as it

rng.seed(12345)
pp = pprint.PrettyPrinter(indent=4)




# to load images

In [2]:
file_ready_for_lines_detection = os.path.join(os.path.dirname(os.path.realpath("__file__")),"phase0-output/processed/88d1c53a-4933-55ab-9223-3d14f4d87a07-e6722290_pg-0_sg-0_M.jpg")
# input_file = os.path.join(os.path.dirname(os.path.realpath("__file__")),"phase0-output/processed/b2432346-2637-5e69-a752-128f71c1bb44-1d8625b3_pg-0_sg-0.jpg")
input_file = os.path.join(os.path.dirname(os.path.realpath("__file__")),"phase0-output/processed/88d1c53a-4933-55ab-9223-3d14f4d87a07-e6722290_pg-0_sg-0_M.jpg")

In [3]:
# img = cv2.imread(file_ready_for_lines_detection)
img = cv2.imread(input_file)
orig_img = cv2.imread(input_file)
line_only_img = cv2.imread(file_ready_for_lines_detection)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
display(Image.fromarray(gray))

In [ ]:
(thresh, img_gray) = cv2.threshold(gray, 128, 255,cv2.THRESH_OTSU)
img_bin = 255-img_gray
display(Image.fromarray(img_bin))

## Trial 1 - sobel x .y

In [ ]:
laplacian = cv2.Laplacian(img_bin,cv2.CV_64F)
sobelx = cv2.Sobel(img_bin,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img_bin,cv2.CV_64F,0,1,ksize=5)

d = Image.fromarray(sobelx)
display(d.convert("L"))

In [ ]:
d = Image.fromarray(sobely)
display(d.convert("L"))

In [ ]:
(thresh, img_bin) = cv2.threshold(sobelx, 128, 255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
img_bin = 255-img_bin 

In [ ]:
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))
print(hori_kernel)
print(verticle_kernel)

In [ ]:
temp_img = cv2.erode(img_bin, hori_kernel, iterations=1)
temp_img2 = cv2.dilate(horizontal_lines_img, hori_kernel, iterations=5)
# morpho close vertical
vertical_lines_img = cv2.erode(img_bin, verticle_kernel, iterations=1)
vertical_lines_img = cv2.dilate(vertical_lines_img, verticle_kernel, iterations=5)

In [ ]:
display(Image.fromarray(vertical_lines_img))

In [ ]:
display(Image.fromarray(horizontal_lines_img))

In [ ]:
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha
# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(vertical_lines_img, alpha, horizontal_lines_img, beta, 0.0)
display((Image.fromarray(img_final_bin)).convert('L'))

In [ ]:
contours, hierarchy = cv2.findContours((Image.fromarray(img_final_bin).convert('L')), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
id= -1
overlay_img = cv2.imread(input_file)
boundRect = [None]*len(contours)
for cnt in contours:
    approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)    
    if len(approx)==4 and cv2.contourArea(cnt) > 100:
#         print(cv2.contourArea(cnt))
        color = (128,35, 125)
        id+=1
#       cv2.drawContours(img,[cnt],0,((id*id)%255,(id*10)%255,255),-1)
        boundRect[id] = cv2.boundingRect(approx)
        cv2.rectangle(overlay_img, (int(boundRect[id][0]), int(boundRect[id][1])),           (int(boundRect[id][0]+boundRect[id][2]), int(boundRect[id][1]+boundRect[id][3])), color, 3) 
        

## Trial 2 - skiimage

In [ ]:
image = img_as_bool(img_bin)
# plt.imshow(image, cmap='gray')

In [ ]:
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))
print(hori_kernel)
print(verticle_kernel)

In [ ]:
horizontal_lines_img = cv2.erode(img_bin, hori_kernel, iterations=5)
horizontal_lines_img = cv2.dilate(horizontal_lines_img, hori_kernel, iterations=5)
# morpho close vertical
vertical_lines_img = cv2.erode(img_bin, verticle_kernel, iterations=5)
vertical_lines_img = cv2.dilate(vertical_lines_img, verticle_kernel, iterations=5)

In [ ]:
display(Image.fromarray(vertical_lines_img))

In [ ]:
display(Image.fromarray(horizontal_lines_img))

In [ ]:
image = horizontal_lines_img
out = ndi.distance_transform_edt(~horizontal_lines_img)
out = out < 0.01 * out.max()
out = morphology.skeletonize(out)
out = morphology.binary_dilation(out, morphology.selem.disk(5))
out = segmentation.clear_border(out)
out = out | image
plt_show(out)

horiz_out = out
image = vertical_lines_img
out = ndi.distance_transform_edt(~vertical_lines_img)
out = out < 0.01 * out.max()
out = morphology.skeletonize(out)
out = morphology.binary_dilation(out, morphology.selem.disk(5))
out = segmentation.clear_border(out)
out = out | image
out = horiz_out | out
# plt_show(out)
# plt.imsave('gaps_filled.jpg', out, cmap='gray')


In [ ]:
import matplotlib as mpl

def plt_show(img, cmap='gray'):
    
    dpi = mpl.rcParams['figure.dpi']
    height, width = img.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(img, cmap)

    plt.show()

## Corner detection techniques

In [ ]:
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 1))

horizontal_lines_img = cv2.erode(img_bin, hori_kernel, iterations=5)
horizontal_lines_img = cv2.dilate(horizontal_lines_img, hori_kernel, iterations=5)
# morpho close vertical
vertical_lines_img = cv2.erode(img_bin, verticle_kernel, iterations=5)
vertical_lines_img = cv2.dilate(vertical_lines_img, verticle_kernel, iterations=5)

img_final_bin = vertical_lines_img | horizontal_lines_img
display((Image.fromarray(img_final_bin)).convert('L'))
# # Weighting parameters, this will decide the quantity of an image to be added to make a new image.
# alpha = 0.5
# beta = 1.0 - alpha
# # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
# img_final_bin = cv2.addWeighted(vertical_lines_img, alpha, horizontal_lines_img, beta, 0.0)
# display((Image.fromarray(com)).convert('L'))

In [ ]:
    dst = cv2.cornerHarris(img_final_bin,2,3,0.04)


In [ ]:
mask = np.zeros_like(img)
mask[dst>0.05*dst.max()]=[0,0,255]
display(Image.fromarray(mask))
coordinates = np.argwhere(mask)
coordinates

In [ ]:
coor_list = [l.tolist() for l in list(coordinates)]
coor_tuples = [tuple(l) for l in coor_list]

In [ ]:
thresh = 15
import math
def distance(pt1, pt2):
    (x1, y1, z), (x2, y2, z) = pt1, pt2
    dist = math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 )
    return dist
save_tuples = coor_tuples.copy()
coor_tuples_copy = coor_tuples

i = 1
removed = 0;
for pt1 in coor_tuples:

    print(' I :', i)
    for pt2  in coor_tuples[i::1]:        
        dist = distance(pt1, pt2)
        print(pt1, pt2, dist)
        if (dist > thresh):
            break;
        
        print('cnt = ' +str(i)+ ', Distance :', dist)
        if(dist < thresh):
            removed +=1
            coor_tuples_copy.remove(pt2)      
        
    i+=1

print("Total points original = "+str(len(save_tuples)))
print("Total points after = "+str(len(coor_tuples_copy)))
print("Total removed = "+str(removed))


In [ ]:
test_img =gray.copy() 
for point in [tuple([y,x]) for x,y,_ in coor_tuples_copy]:
        cv2.circle(test_img, point,5, (0,128,128), 0)

plt_show(test_img)

In [ ]:
x_tuples = [tuple([x,y]) for y,x,_ in coor_tuples]
y_tuples = [tuple([y,x]) for y,x,_ in coor_tuples]

x_dict = {k:tuple(x[1] for x in v) for k,v in it.groupby(sorted(x_tuples), key=lambda x: x[0])}
y_dict = {k:tuple(x[1] for x in v) for k,v in it.groupby(sorted(y_tuples), key=lambda x: x[0])}

pp.pprint(x_dict)
pp.pprint(y_dict)

# get_adjacent_nearest_points(points_dict, None)

In [ ]:
only_point_img = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
for key in y_dict:    
    cv2.circle(only_point_img, (int(y_dict[key][0]), int(key)), 5, (200,200,128), 5)

display(Image.fromarray(only_point_img))

### Getting the nearest polar coordinates 
1. for a given x...lookup all y points and then get the shorest distance
2. for a given y ... lookup all x points and then get the shortest distance

In [ ]:
def get_adjacent_nearest_points(target_point):
    
    polr_cords = []
    
    pp.pprint(points_dict)

#### drawing lines over non-rectangles 

In [ ]:
(ret, thresh) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
 # Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

id= -1
boundRect = [None]*len(contours)
overlay_img = cv2.cvtColor(test_img,cv2.COLOR_GRAY2RGB)
for cnt in contours:
    approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)    
    if len(approx)<=3 and cv2.contourArea(cnt) > 5:
#         print(cv2.contourArea(cnt))
        color = ((245+id*2 % 255), 90+id*2, 66)
        id+=1
#       cv2.drawContours(img,[cnt],0,((id*id)%255,(id*10)%255,255),-1)
        boundRect[id] = cv2.boundingRect(approx)
        cv2.rectangle(overlay_img, (int(boundRect[id][0]), int(boundRect[id][1])), \
          (int(boundRect[id][0]+boundRect[id][2]), int(boundRect[id][1]+boundRect[id][3])), color, 4) 
        
#         x, y, w, h = cv2.boundingRect(cnt) 
#         r = {'x':x,'y':y,'w':w,'h':h}        
#         bounding_rects.append(SimpleNamespace(**r))
    elif (len(approx) > 4):
#         print("poly length = "+str(approx))
#         pp.pprint(approx)
        cv2.polylines(overlay_img, approx,True, color, 4) 
#         x,y,w,h = cv2.boundingRect(approx)
#         cv2.rectangle(overlay_img,(x,y),(x+w,y+h), color, 4) 
        pp.pprint(cv2.boundingRect(approx))
        print("---------")
        

# # img = line_only_img



In [ ]:
display(Image.fromarray(overlay_img))

In [ ]:
def sort_contours(cnts, method="left-to-right"):
        # initialize the reverse flag and sort index
        reverse = False
        i = 0
    
        # handle if we need to sort in reverse
        if method == "right-to-left" or method == "bottom-to-top":
            reverse = True
    
        # the x-coordinate of the bounding box
        if method == "top-to-bottom" or method == "bottom-to-top":
            i = 1
    
        # construct the list of bounding boxes and sort them from top to
        # bottom
        boundingBoxes = [cv2.boundingRect(c) for c in cnts]
        (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
            key=lambda b:b[1][i], reverse=reverse))
    
        # return the list of sorted contours and bounding boxes
        return (cnts, boundingBoxes)

In [ ]:
def sort_poly(p):
    min_axis = np.argmin(np.sum(p, axis=1))
    p = p[[min_axis, (min_axis+1)%4, (min_axis+2)%4, (min_axis+3)%4]]
    if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
        return p
    else:
        return p[[0, 3, 2, 1]]